# Библиотеки

In [760]:
import numpy as np
from random import shuffle
from PIL.Image import open
from os import listdir
from typing import List

# Класс нейронки

In [761]:
class KN:
    lr: float
    D: float = 1
    

    def __init__(self, input, clasters): 
        self.weights = np.random.uniform(low=-0.3, high=0.3, size=(clasters, input))

    def predict(self, vector: np.ndarray):
        dist: np.ndarray =  np.power((vector - self.weights), 2).sum(axis=1)
        winner_index = dist.argmin()
        return winner_index

    def train(self, vector: np.ndarray):
        winner_index = self.predict(vector)

        all_dists: np.ndarray = np.zeros(5, dtype=np.float32)
        rows, _ = self.weights.shape
        for index in range(0, rows):
            if index == winner_index:
                continue
            else:
                all_dists[index] = (np.power((vector - self.weights[index]), 2).sum())

        if self.D is None:
            max_dist_index = all_dists.argmax()
            self.D = all_dists[max_dist_index]
        
        all_errors = []
        for index in range(0, len(all_dists)):
            if index == winner_index or all_dists[index] < kn.D:
                delta: np.ndarray = self.lr * (vector - self.weights[index])
                self.weights[index] += delta
                all_errors.append(np.abs(delta))
            
        all_errors = np.array(all_errors)
        return all_errors.sum()

# Загрузка датасета

In [762]:
def normalize(image: np.ndarray):
    new_image = []
    for rgb in image:
        rgb: np.ndarray
        if (rgb == [255,255,255]).all():
            new_image.append(0)
        else:
            new_image.append(1)

    return np.array(new_image)

In [763]:
dataset: List[tuple] = []
for file in listdir('data'):
    image = np.array(open(f'data/{file}'))
    x_max, y_max, _ = image.shape
    image = image.reshape((x_max*y_max, 3))
    image = normalize(image)
    dataset.append(tuple((file, image)))

In [764]:
test: List[tuple] = []
for file in listdir('test'):
    image = np.array(open(f'test/{file}'))
    x_max, y_max, _ = image.shape
    image = image.reshape((x_max*y_max, 3))
    image = normalize(image)
    test.append(tuple((file, image)))

# Обучение

In [765]:
kn = KN(2500, 5)
epoch = 200
kn.lr = 0.8

all_deltas = []
epoch_count = 0
error_counter = np.zeros(shape=5)

for i in range(epoch):
    shuffle(dataset)
    delta: float = 0
    for _, image in dataset:
        delta += kn.train(image)
    
    delta = delta / len(dataset)
    all_deltas.append(round(delta, 5))
    if (delta < 0.05): break

    epoch_count += 1
    kn.lr *= 0.9
    kn.D *= 0.9

# Классы

In [766]:
print('Обучающая выборка:')
all_class = { 0: {}, 1: {}, 2: {}, 3: {}, 4: {} }
for filename, image in dataset:
    classes = kn.predict(image)
    default_value = all_class[classes].get(filename.split(' ')[0], 0)
    new_value = default_value + 1
    all_class[classes][filename.split(' ')[0]] = new_value
    print(f'{filename}: Класс {classes}')

correct_classes = {}
for _class in range(0, len(all_class)):
    max_key = max(all_class[_class], key=all_class[_class].get)
    correct_classes[_class] = max_key
print(correct_classes)

print(f'\nИзменения на эпохе {all_deltas}')
print(f'Прошло эпох: {epoch_count}')

print('Тестовая выборка:')
error = 0
for filename, image in test:
    classes = kn.predict(image)
    if filename.split(' ')[0] != correct_classes[classes]: error += 1  
    print(f'{filename}: Класс {classes}')
    
        

print(f'Ошибка на тестовой выборке: {error / len(test)}')

Обучающая выборка:
плюс (12).png: Класс 2
плюс (4).png: Класс 2
крест (6).png: Класс 4
буква (6).png: Класс 0
плюс (8).png: Класс 2
плюс (1).png: Класс 2
буква (3).png: Класс 0
смайлик (16).png: Класс 1
крест (1).png: Класс 3
буква (7).png: Класс 0
плюс (14).png: Класс 2
крест (15).png: Класс 4
буква (14).png: Класс 0
буква (9).png: Класс 0
круг (6).png: Класс 1
буква (8).png: Класс 0
смайлик (12).png: Класс 1
буква (5).png: Класс 0
крест (16).png: Класс 4
смайлик (18).png: Класс 1
круг (10).png: Класс 1
смайлик (9).png: Класс 1
смайлик (2).png: Класс 1
крест (4).png: Класс 3
плюс (20).png: Класс 2
круг (16).png: Класс 1
крест (3).png: Класс 3
смайлик (6).png: Класс 1
смайлик (4).png: Класс 1
плюс (17).png: Класс 2
круг (1).png: Класс 1
буква (12).png: Класс 0
круг (13).png: Класс 1
круг (7).png: Класс 1
плюс (10).png: Класс 2
смайлик (1).png: Класс 1
смайлик (11).png: Класс 1
круг (17).png: Класс 1
крест (17).png: Класс 4
крест (5).png: Класс 3
крест (11).png: Класс 4
круг (15).png: К